In [46]:
import pandas as pd

In [47]:
df=pd.read_csv('iot23_combined.csv')
df

In [48]:


# Import necessary libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [49]:
del df['Unnamed: 0']
del df['uid']

In [50]:
df

In [51]:
df.columns

In [52]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Sample dataframe (replace this with your actual data)
# df = pd.read_csv('your_data.csv')  # Load your actual dataset

# List of columns you want to encode (adjust as necessary)
columns_to_encode = ['proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 
                     'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 
                     'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 
                     'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 
                     'conn_state_SF', 'conn_state_SH', 'conn_state_SHR', 
                     'service_-', 'service_dhcp', 'service_dns', 'service_http', 
                     'service_irc', 'service_ssh', 'service_ssl']

# Loop through each column and apply label encoding for True/False
for column in columns_to_encode:
    df[column] = df[column].apply(lambda x: 1 if x == True else 0)

# Check the first few rows of the dataframe after encoding
print(df.head())


In [53]:
df.columns

In [54]:
# Drop the 'ts' and 'missed_bytes' columns
df = df.drop(['ts', 'missed_bytes'], axis=1)

# Check the first few rows of the dataframe after dropping the columns
print(df.head())


In [55]:
from sklearn.utils import resample
import pandas as pd

df = df[~df['label'].isin(['Okiru-Attack', 'C&C-Mirai'])]
# # Assuming `df_c` is your dataset
# # Step 1: Identify the majority and minority classes
# majority_class = 'PartOfAHorizontalPortScan'  # Replace with your majority class label
# minority_classes = df['label'].unique()
# minority_classes = [cls for cls in minority_classes if cls != majority_class]

# # Step 2: Downsample the majority class
# df_majority = df[df['label'] == majority_class]
# df_minority = df[df['label'] != majority_class]

# # Determine the target size (based on the largest minority class)
# target_size = df_minority['label'].value_counts().max()

# df_majority_downsampled = resample(df_majority,
#                                    replace=False,    # Sample without replacement
#                                    n_samples=target_size,  # Match minority size
#                                    random_state=42)

# # Step 3: Combine downsampled majority class with the minority classes
# df_balanced = pd.concat([df_majority_downsampled, df_minority])

# # Step 4: Shuffle the dataset
# df = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# # Display the balanced and shuffled dataset
print(df['label'].value_counts())


In [56]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Label encode the 'label' column
df['label'] = label_encoder.fit_transform(df['label'])

label_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}

# Check the first few rows of the dataframe after encoding
print(df.head())


In [57]:
print(label_mapping)

In [58]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
numerical_features = ['orig_bytes', 'resp_bytes', 'duration', 'orig_pkts', 'resp_pkts']
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [59]:
df['label'].value_counts()

In [60]:
# Import necessary libraries
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming 'df' is your dataframe with 'label' column as the target variable
X = df.drop('label', axis=1)  # Features (excluding the label column)
y = df['label']  # Label column

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, random_state=42)

# Initialize XGBoost Classifier
xgb_model = XGBClassifier(random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model (accuracy score)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Optionally, check feature importance
print("Feature importances:", xgb_model.feature_importances_)


In [61]:
validation_data = pd.concat([pd.DataFrame(X_val), pd.DataFrame(y_val, columns=['label'])], axis=1)

validation_data.to_csv("validation_dataset.csv",index=False)

In [62]:

# Initialize RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Optionally, you can check feature importance
print("Feature importances:", rf_model.feature_importances_)

In [63]:
# Import necessary libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming 'df' is your dataframe with 'label' column as the target variable


# Initialize DecisionTreeClassifier
dt_model = DecisionTreeClassifier(random_state=42)

# Train the model
dt_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = dt_model.predict(X_test)

# Evaluate the model (accuracy score)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Optionally, check feature importances
print("Feature importances:", dt_model.feature_importances_)


In [64]:
df.shape


In [65]:
df['label'].unique()

In [66]:
import joblib

# Save the trained XGBoost model to a file
joblib.dump(xgb_model, "xgb_model.pkl")
print("Model saved to xgb_model.pkl")


In [67]:
# Load the saved XGBoost model
loaded_model = joblib.load("xgb_model.pkl")

# Use the loaded model for predictions or further evaluation
y_pred_loaded = loaded_model.predict(X_test)
accuracy_loaded = accuracy_score(y_test, y_pred_loaded)
print(f'Accuracy with loaded model: {accuracy_loaded * 100:.2f}%')


In [68]:
df1=pd.read_csv('validation_dataset.csv')
df1

In [69]:
df1['label'].value_counts()